<a href="https://colab.research.google.com/github/ykalathiya-2/unsloath/blob/main/unsloath_LoRA_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ⚠️ IMPORTANT: Preventing NaN Loss

**This notebook has been updated to prevent NaN (Not a Number) loss issues.**

### Key Changes Made:
1. ✅ **Learning Rate**: Reduced from `2e-4` to `5e-5` (safer for small models)
2. ✅ **Warmup Steps**: Increased from `5` to `10` (smoother start)
3. ✅ **Gradient Clipping**: Added `max_grad_norm = 1.0` (prevents explosion)

### If You Still Get NaN:
- Check the troubleshooting section near the training cell
- Consider using FP32 instead of FP16 (more stable but slower)
- Further reduce learning rate to `1e-5`

**These settings are now optimized for SmolLM2-135M!** 🚀

---

In [1]:
%%capture
import os
!pip install --upgrade -qqq uv
if "COLAB_" not in "".join(os.environ.keys()):
    # If you're not in Colab, just use pip install!
    !pip install unsloth vllm synthetic-data-kit==0.0.3
else:
    try: import numpy, PIL; get_numpy = f"numpy=={numpy.__version__}"; get_pil = f"pillow=={PIL.__version__}"
    except: get_numpy = "numpy"; get_pil = "pillow"
    try: import subprocess; is_t4 = "Tesla T4" in str(subprocess.check_output(["nvidia-smi"]))
    except: is_t4 = False
    get_vllm, get_triton = ("vllm==0.9.2", "triton==3.2.0") if is_t4 else ("vllm==0.10.2", "triton")
    !uv pip install -qqq --upgrade         unsloth {get_vllm} {get_numpy} {get_pil} torchvision bitsandbytes xformers
    !uv pip install -qqq {get_triton}
    !uv pip install synthetic-data-kit==0.0.3
!uv pip install transformers==4.56.2
!uv pip install --no-deps trl==0.22.2

In [2]:
from unsloth.dataprep import SyntheticDataKit

generator = SyntheticDataKit.from_pretrained(
    # Choose any model from https://huggingface.co/unsloth
    model_name = "unsloth/Llama-3.2-3B-Instruct",
    max_seq_length = 2048, # Longer sequence lengths will be slower!
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
INFO 11-10 08:41:42 [__init__.py:216] Automatically detected platform cuda.
🦥 Unsloth Zoo will now patch everything to make training faster!


config.json:   0%|          | 0.00/890 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

INFO 11-10 08:41:55 [vllm_utils.py:700] Unsloth: Patching vLLM v1 graph capture
INFO 11-10 08:41:55 [vllm_utils.py:730] Unsloth: Patching vLLM v0 graph capture
Unsloth: Using dtype = torch.bfloat16 for vLLM.
Unsloth: vLLM loading unsloth/Llama-3.2-3B-Instruct with actual GPU utilization = 89.25%
Unsloth: Your GPU has CUDA compute capability 8.9 with VRAM = 22.16 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 2048. Num Sequences = 256.
Unsloth: vLLM's KV Cache can use up to 13.79 GB. Also swap space = 4 GB.
Unsloth: Disabling `disable_cascade_attn` in vLLM to allow for better on policy RL!
Unsloth: Not an error, but `device` is not supported in vLLM. Skipping.
vLLM STDOUT: INFO 11-10 08:42:08 [__init__.py:216] Automatically detected platform cuda.
vLLM STDOUT: (APIServer pid=1795) INFO 11-10 08:42:10 [api_server.py:1896] vLLM API server version 0.10.2
vLLM STDOUT: (APIServer pid=1795) INFO 11-10 08:42:10 [utils.py:328] non-default args: {'model_tag': 'unsloth/Llama-

## Generate QA Pairs + Auto clean data
We now use synthetic data kit for question answer pair generation:

In [3]:
generator.prepare_qa_generation(
    output_folder = "data", # Output location of synthetic data
    temperature = 0.7, # Higher temp makes more diverse datases
    top_p = 0.95,
    overlap = 64, # Overlap portion during chunking
    max_generation_tokens = 512, # Can increase for longer QA pairs
)

Check if it succeeded:

In [4]:
!synthetic-data-kit system-check

vLLM STDOUT: (APIServer pid=1795) INFO:     127.0.0.1:37254 - "GET /v1/models HTTP/1.1" 200 OK
 VLLM server is running at http://localhost:8000/v1
Available models: {'object': 'list', 'data': [{'id': 
'unsloth/Llama-3.2-3B-Instruct', 'object': 'model', 'created': 1762764231, 
'owned_by': 'vllm', 'root': 'unsloth/Llama-3.2-3B-Instruct', 'parent': None, 
'max_model_len': 2048, 'permission': [{'id': 
'modelperm-d2976bdc3602495486161fd8c0084351', 'object': 'model_permission', 
'created': 1762764231, 'allow_create_engine': False, 'allow_sampling': True, 
'allow_logprobs': True, 'allow_search_indices': False, 'allow_view': True, 
'allow_fine_tuning': False, 'organization': '*', 'group': None, 'is_blocking': 
False}]}]}
⠋ Checking VLLM server at http://localhost:8000/v1...


## Document Parsing (PDF, CSV, HTML etc.)
Now, let's take the Byte Latent Transformer: Patches Scale Better Than Tokens research paper found at https://arxiv.org/abs/2412.09871 and covert it to Q&A pairs in order to finetune Llama 3.2!

In [5]:
# Byte Latent Transformer: Patches Scale Better Than Tokens paper in HTML format
!synthetic-data-kit \
    -c synthetic_data_kit_config.yaml \
    ingest "https://arxiv.org/html/2412.09871v1"

# Truncate document
filenames = generator.chunk_data("data/output/arxiv_org.txt")
print(len(filenames), filenames[:3])

⠧ Processing https://arxiv.org/html/2412.09871v1...
 Text successfully extracted to data/output/arxiv_org.txt
34 ['data/output/arxiv_org_0.txt', 'data/output/arxiv_org_1.txt', 'data/output/arxiv_org_2.txt']


We see around 37 chunks of data. We now call synthetic-data-kit to create some pairs of data for 3 of our chunks.

You can process more chunks, but it'll be much slower!

Using `--num-pairs` will generate **approximately** that many QA pairs. However it might be shorter or longer depending on the `max_seq_length` of the loaded up model. So if you specify 100, you might only get 10 since the model's max sequence length is capped.

In [6]:
import time
# Process 3 chunks for now -> can increase but slower!
for filename in filenames[:3]:
    !synthetic-data-kit \
        -c synthetic_data_kit_config.yaml \
        create {filename} \
        --num-pairs 25 \
        --type "qa"
    time.sleep(2) # Sleep some time to leave some room for processing

vLLM STDOUT: (APIServer pid=1795) INFO:     127.0.0.1:58944 - "GET /v1/models HTTP/1.1" 200 OK
vLLM STDOUT: (APIServer pid=1795) INFO:     127.0.0.1:58956 - "GET /v1/models HTTP/1.1" 200 OK
⠴ Generating qa content from data/output/arxiv_org_0.txt...vLLM STDOUT: (APIServer pid=1795) INFO 11-10 08:43:54 [chat_utils.py:538] Detected the chat template content format to be 'string'. You can set `--chat-template-content-format` to override this.
⠏ Generating qa content from data/output/arxiv_org_0.txt...vLLM STDOUT: (APIServer pid=1795) INFO:     127.0.0.1:58972 - "POST /v1/chat/completions HTTP/1.1" 200 OK
Processing 1 chunks to generate QA pairs...
⠦ Generating qa content from data/output/arxiv_org_0.txt...vLLM STDOUT: (APIServer pid=1795) INFO:     127.0.0.1:58988 - "POST /v1/chat/completions HTTP/1.1" 200 OK
Batch processing complete.
Generated 11 QA pairs total
Saving result to data/generated/arxiv_org_0_qa_pairs.json
Successfully wrote test file to data/generated/test_write.json
Succes

We now convert the generated datasets into QA formats so we can load it for finetuning:

In [7]:
qa_pairs_filenames = [
    f"data/generated/arxiv_org_{i}_qa_pairs.json"
    for i in range(len(filenames[:3]))
]
for filename in qa_pairs_filenames:
    !synthetic-data-kit \
        -c synthetic_data_kit_config.yaml \
        save-as {filename} -f ft

⠋ Converting data/generated/arxiv_org_0_qa_pairs.json to ft format with json 
storage...
 Converted to ft format and saved to data/final/arxiv_org_0_qa_pairs_ft.json
⠋ Converting data/generated/arxiv_org_1_qa_pairs.json to ft format with json 
storage...
 Converted to ft format and saved to data/final/arxiv_org_1_qa_pairs_ft.json
⠋ Converting data/generated/arxiv_org_2_qa_pairs.json to ft format with json 
storage...
 Converted to ft format and saved to data/final/arxiv_org_2_qa_pairs_ft.json


Let's load up the data and see what the synthetic data looks like!

In [8]:
from datasets import Dataset
import pandas as pd
final_filenames = [
    f"data/final/arxiv_org_{i}_qa_pairs_ft.json"
    for i in range(len(filenames[:3]))
]
conversations = pd.concat([
    pd.read_json(name) for name in final_filenames
]).reset_index(drop = True)

dataset = Dataset.from_pandas(conversations)

In [9]:
dataset[-1]

{'messages': [{'content': 'You are a helpful assistant.', 'role': 'system'},
  {'content': 'What are the three patching functions introduced in this work?',
   'role': 'user'},
  {'content': 'Patching with a fixed number of bytes per patch, entropy-based patching, and patching on entropy using a small CNN byte-level model with 2-byte context.',
   'role': 'assistant'}]}

Finally free vLLM process to save memory and to allow for finetuning!

In [10]:
generator.cleanup()

Attempting to terminate the VLLM server gracefully...
vLLM STDOUT: (APIServer pid=1795) WARNING 11-10 08:45:01 [launcher.py:98] port 8000 is used by process psutil.Process(pid=1795, name='vllm', status='running') launched with command:
vLLM STDOUT: (APIServer pid=1795) WARNING 11-10 08:45:01 [launcher.py:98] /usr/bin/python3 /usr/local/bin/vllm serve unsloth/Llama-3.2-3B-Instruct --gpu-memory-utilization 0.8924844251775063 --max-model-len 2048 --load-format auto --kv-cache-dtype auto --dtype bfloat16 --max-num-batched-tokens 2048 --max-num-seqs 256 --max-logprobs 0 --seed 0 --max-lora-rank 16 --max-loras 1 --disable-log-stats --enable-prefix-caching --enable-chunked-prefill --swap-space 4 --disable-cascade-attn
vLLM STDOUT: (APIServer pid=1795) INFO 11-10 08:45:01 [launcher.py:101] Shutting down FastAPI HTTP server.
Server terminated gracefully.


### Fine-tuning Synthetic Dataset with Unsloth

In [11]:
from unsloth import FastLanguageModel
import torch
from google.colab import userdata

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "HuggingFaceTB/SmolLM2-135M",
    max_seq_length = 2048, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    token = userdata.get('HUGGING_FACE_TOKEN'), # use one if using gated models
    torch_dtype=torch.float16,
)

==((====))==  Unsloth 2025.11.2: Fast Llama patching. Transformers: 4.56.2. vLLM: 0.10.2.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/831 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

HuggingFaceTB/SmolLM2-135M does not have a padding token! Will use pad_token = <|endoftext|>.


In [12]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.11.2 patched 30 layers with 30 QKV layers, 30 O layers and 30 MLP layers.


<a name="Data"></a>
### Data Prep
We now use the `Llama-3.2` format for conversation style finetunes. The chat template renders conversations like below: (Cutting Knowledge Date is by default there!)

```
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 01 May 2025

You are a helpful assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>

What is 1+1?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

2<|eot_id|>
```

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [13]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

Model does not have a padding token! Will use pad_token = <|endoftext|>.


In [14]:
def formatting_prompts_func(examples):
    convos = examples["messages"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }

# Get our previous dataset and format it:
dataset = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/33 [00:00<?, ? examples/s]

See result of the first row:

In [15]:
dataset[0]

{'messages': [{'content': 'You are a helpful assistant.', 'role': 'system'},
  {'content': 'What is the primary units of computation in the Byte Latent Transformer (BLT) architecture?',
   'role': 'user'},
  {'content': 'patches', 'role': 'assistant'}],
 'text': '<|endoftext|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\nYou are a helpful assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWhat is the primary units of computation in the Byte Latent Transformer (BLT) architecture?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\npatches<|eot_id|>'}

<a name="Train"></a>
### Train the model
Now let's train our model. We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`.

In [ ]:
from trl import SFTTrainer, SFTConfig

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = None, # Can set up evaluation!
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        warmup_steps = 10,               # ✅ Increased from 5 (prevents NaN)
        max_steps = 30,
        learning_rate = 5e-5,            # ✅ Reduced from 2e-4 (prevents NaN)
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.001,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use TrackIO/WandB etc
        max_grad_norm = 1.0,             # ✅ Gradient clipping (prevents NaN)
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=16):   0%|          | 0/33 [00:00<?, ? examples/s]

### 🔍 Pre-Training Diagnostics

Before training, let's verify our data is properly formatted:

In [ ]:
# Verify dataset quality before training
print("📊 Dataset Statistics:")
print(f"   Total samples: {len(dataset)}")
print(f"   Sample format check:")

# Check first sample
sample = dataset[0]
print(f"   - Has 'text' field: {'text' in sample}")
print(f"   - Text length: {len(sample['text'])} characters")
print(f"   - First 200 chars: {sample['text'][:200]}...")

# Check for potential issues
print("\n🔍 Potential Issues Check:")
has_none = any(x['text'] is None for x in dataset)
has_empty = any(len(x['text']) == 0 for x in dataset)
print(f"   - None values: {has_none} {'❌ FIX NEEDED' if has_none else '✅'}")
print(f"   - Empty strings: {has_empty} {'❌ FIX NEEDED' if has_empty else '✅'}")

# Token length statistics
sample_lengths = []
for i in range(min(10, len(dataset))):
    tokens = tokenizer(dataset[i]['text'], return_tensors="pt", truncation=False)
    sample_lengths.append(len(tokens['input_ids'][0]))

print(f"\n📏 Token Length Statistics (first 10 samples):")
print(f"   - Min: {min(sample_lengths)} tokens")
print(f"   - Max: {max(sample_lengths)} tokens")
print(f"   - Average: {sum(sample_lengths)/len(sample_lengths):.1f} tokens")
if max(sample_lengths) > 2048:
    print(f"   ⚠️  Warning: Some samples exceed max_seq_length (2048)")

print("\n✅ Data looks good! Ready to train.")

In [17]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA L4. Max memory = 22.161 GB.
0.193 GB of memory reserved.


In [18]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 33 | Num Epochs = 6 | Total steps = 30
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 4,884,480 of 139,399,488 (3.50% trained)


Step,Training Loss
1,3.601600
2,nan
3,nan
4,nan
5,nan
6,nan
7,nan
8,nan
9,nan
10,nan


## ⚠️ TROUBLESHOOTING: NaN Loss Issue

**If you see NaN loss values**, it means training became unstable. Here are the fixes:

### Common Causes & Solutions:

1. **Learning Rate Too High** ⭐ MOST COMMON
   - **Problem**: `2e-4` is too aggressive for small models
   - **Solution**: Reduce to `5e-5` or `1e-4`
   
2. **Insufficient Warmup Steps**
   - **Problem**: 5 warmup steps not enough
   - **Solution**: Increase to 10-20 steps
   
3. **Mixed Precision Issues**
   - **Problem**: FP16 can cause numerical instability
   - **Solution**: Add `fp16 = False, bf16 = False` to use FP32
   
4. **Gradient Clipping**
   - **Problem**: Gradients exploding
   - **Solution**: Add `max_grad_norm = 1.0`

### 🔧 Fixed Training Configuration:

If you got NaN, **restart the kernel** and modify the trainer configuration:

```python
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = None,
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 10,              # ✅ Increased warmup
        max_steps = 30,
        learning_rate = 5e-5,            # ✅ Reduced LR (was 2e-4)
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.001,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none",
        max_grad_norm = 1.0,             # ✅ Gradient clipping
        fp16 = False,                    # ✅ Use FP32 for stability
        bf16 = False,
    ),
)
```

### Alternative: More Conservative Settings

For maximum stability (slower but more reliable):

```python
args = SFTConfig(
    dataset_text_field = "text",
    per_device_train_batch_size = 1,     # Smaller batch
    gradient_accumulation_steps = 8,     # Maintain effective batch size
    warmup_steps = 15,
    max_steps = 30,
    learning_rate = 1e-5,                # Very conservative
    logging_steps = 1,
    optim = "adamw_8bit",
    weight_decay = 0.01,
    lr_scheduler_type = "cosine",        # Smoother decay
    seed = 3407,
    report_to = "none",
    max_grad_norm = 0.5,                 # Stricter clipping
    fp16 = False,
    bf16 = False,
)
```

### 📊 What to Expect After Fix:

✅ **Good Loss Values**: 2.5 → 2.0 → 1.5 (decreasing)  
❌ **Bad Loss Values**: 3.6 → NaN (training crashed)

### 🔄 To Continue:

1. **Runtime → Restart Runtime** (in Colab)
2. Re-run all cells from the beginning
3. Use the fixed configuration above
4. Training should now be stable!

In [19]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

45.7412 seconds used for training.
0.76 minutes used for training.
Peak reserved memory = 0.26 GB.
Peak reserved memory for training = 0.067 GB.
Peak reserved memory % of max memory = 1.173 %.
Peak reserved memory for training % of max memory = 0.302 %.


In [20]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

45.7412 seconds used for training.
0.76 minutes used for training.
Peak reserved memory = 0.26 GB.
Peak reserved memory for training = 0.067 GB.
Peak reserved memory % of max memory = 1.173 %.
Peak reserved memory for training % of max memory = 0.302 %.


<a name="Inference"></a>
### Inference
Let's run the model! Use `apply_chat_template` with `add_generation_prompt` set to `True` for inference.

In [21]:
messages = [
    {"role": "user", "content": "What is the Byte Latent Transformer?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer,
                   max_new_tokens = 256, temperature = 0.1)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|endoftext|>


The model learns about the research paper!!

In [22]:
messages = [
    {"role": "user", "content": "What are some benefits of the BLT?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer,
                   max_new_tokens = 256, temperature = 0.1)

<|endoftext|>


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [23]:
model.save_pretrained("SmolLM2-135M_LoRA_finetuned")  # Local saving
tokenizer.save_pretrained("SmolLM2-135M_LoRA_finetuned")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('SmolLM2-135M_LoRA_finetuned/tokenizer_config.json',
 'SmolLM2-135M_LoRA_finetuned/special_tokens_map.json',
 'SmolLM2-135M_LoRA_finetuned/chat_template.jinja',
 'SmolLM2-135M_LoRA_finetuned/vocab.json',
 'SmolLM2-135M_LoRA_finetuned/merges.txt',
 'SmolLM2-135M_LoRA_finetuned/added_tokens.json',
 'SmolLM2-135M_LoRA_finetuned/tokenizer.json')